## Viz complémentaire axe 2.3

Silvia 16/05 - "Thierry, que en penses tu de demander une carte des destinations de navires chargés du sel (tonnage total?) sortis de Marennes pour explorer la question des marchés et des acteurs ?
avec possibilité de choisir pavillon français / pavillon étranger ?"

Nous utilisons des données Navigo :
- pointcalls Navigo : pointcal_action = 'Out', year = 1789, ferme_bureau = 'Marennes', et regarder ship_flag_standardized_fr pour le pavillon

##### que puis je faire pour avoir la bonne destination ? à priori je ne peux pas utiliser les flows pour 1789
**=> solution choisie : choisir pointcalls In qui ont le même source_doc_id que les pointcalls Out de Marennes**

Intéressant de regarer flows Toflit aussi ? si oui on pourrait utiliser
- flows Toflit : year = 1789, customs_office = 'Marennes', export_import = 'Export'
(pour l'instant je ne l'ai pas fait, principalement à cause du problème qu'on n'a pas de pavillon de bateau à proprement parler : on pourrait regarder 'partner_revolutionempire' mais ce n'est pas le pavillon à proprement parler)

#### Outputs en bas de ce notebook :
#### 1. Réseau des destinations : 1ere représentation des destinations des navires chargés de Sel, sans distinguer flux relevants de bateaux au pavillon français / étranger
#### 2. Carte des flux dont la taille dépend du tonnage cumulé, distinguant pavillons français (bleu) / étrangers (orange), en schématisant les départs sur le port de Marennes même

In [ ]:
# imports
import json 
from poitousprint import Portic, Toflit, combine_commodity_purposes, get_pointcalls_commodity_purposes_as_toflit_product

portic_client = Portic()
toflit_client = Toflit()

In [ ]:
pointcalls_navigo = portic_client.get_pointcalls(
    year = 1789, 
    ferme_bureau = 'Marennes', # après vérification, tous les pointcalls 'Out' de 1789 qui concernent les ports rattachés au bureau des Fermes de Marennes ont bien l'attribut ferme_bureau à 'Marennes'
    pointcall_action = 'Out'
)

print("nombre de pointcalls sortant de Marennes avec du sel :", len(pointcalls_navigo))

In [ ]:
pointcall_test = pointcalls_navigo[0] 
# verifier ce à quoi peut ressembler un pointcall correspondant
combine_commodity_purposes(pointcall_test)

In [ ]:
pointcalls_navigo_with_revolution_empire_products = get_pointcalls_commodity_purposes_as_toflit_product(pointcalls_navigo, product_classification="product_revolutionempire")
# print(pointcalls_navigo_with_revolution_empire_products[100:110])

In [ ]:
# filtrer les pointcalls pour lesquels la cargaison contient du sel
final_pointcalls_navigo = []
final_pointcalls_source_doc_ids = [] # va contenir tous les pkids des pointcalls Out dont on veut trouvé la destination => pointcall In avec le même pkid

for pointcall in pointcalls_navigo_with_revolution_empire_products:
    for commodity_purpose in pointcall['commodity_purposes']:
        if (commodity_purpose['commodity_as_toflit'] == 'Sel'):
            final_pointcalls_navigo.append(pointcall)
            final_pointcalls_source_doc_ids.append(pointcall['source_doc_id'])
        
print("Nombre de pointcalls pertinents trouvés ('Out' de Marennes en 1789 et transportant du sel) :", len(final_pointcalls_navigo))

# print("\nExemple de source_doc_id récupérés : ", final_pointcalls_source_doc_ids[0:20])
# print("\nExemple de pointcall pertinent : ", final_pointcalls_navigo[0])

In [ ]:
# retrouver les destinations
pointcalls_navigo_in = portic_client.get_pointcalls(
    start_year = 1789,
    end_year = 1790, # on considère que le voyage ne dure pas plus d'un an
    pointcall_action = ['In', 'in'] # peut être encore des soucis de casse dans l'API 
)

print("Nombre de pointcalls In trouvés entre 1789 et 1790 :", len(pointcalls_navigo_in))

# print("\nExemple de pointcall pertinent : ", pointcalls_navigo_in[0])

In [ ]:
# filtrer les pointcalls In considérés comme destination des Out sortant de Marennes avec du sel (même pkid)
final_pointcalls_in = []

for pointcall in pointcalls_navigo_in:
    if (str(pointcall['source_doc_id']) in final_pointcalls_source_doc_ids):
        final_pointcalls_in.append(pointcall)

print("Nombre de pointcalls pertinents trouvés ('In' : destination des sorties de sel de Marennes en 1789) :", len(final_pointcalls_in))

# print("\nExemple de pointcall pertinent : ", final_pointcalls_in[0])

In [ ]:
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

import networkx as nx

def build_dep_dest_graph(data_dep, data_dest, key_1, key_2, **kwargs):
    """
    Cette fonction prend un ensemble de dict et deux noms de clés.
    Elle renvoie un graphe networkx de coocurrence entre les dicts
    """
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 255, 0)",
        "color_2": "rgb(255, 0, 0)",
        "node_min_size": 1,
        "node_max_size": 10
    }
    params = {
        **default_params,
        **kwargs
    }

    # remplir les dicts
    for datum_dep in data_dep:
        for datum_dest in data_dest :
            if (key_1 in datum_dep and key_2 in datum_dest and datum_dep['source_doc_id'] == datum_dest['source_doc_id']):
                value_1 = datum_dep[key_1] if datum_dep[key_1] is not None else "undefined"
                value_2 = datum_dest[key_2] if datum_dest[key_2] is not None else "undefined"
                value_1_id = key_1 + "_" + value_1
                value_2_id = key_2 + "_" + value_2
                if value_1_id in key1_uniq:
                    key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
                else:
                   key1_uniq[value_1_id] = {
                       "type": key_1,
                       "name": value_1,
                       "color": params["color_1"],
                       "size": 1
                   }
                if value_2_id in key2_uniq:
                    key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
                else:
                   key2_uniq[value_2_id] = {
                       "type": key_2,
                       "name": value_2,
                       "color": params["color_2"],
                       "size": 1
                   }
                edge_footprint = value_1_id + "-" + value_2_id
                if edge_footprint in edges_uniq:
                    edges_uniq[edge_footprint]["weight"] += 1
                else:
                    edges_uniq[edge_footprint] = {
                        "source": value_1_id,
                        "target": value_2_id,
                        "weight": 1
                    }
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))

    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [params["node_min_size"], params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]

    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Graph

In [ ]:
# 1ere représentation des destinations des navires chargés de Sel, sans distinguer flux relevants de bateaux au pavillon français / étranger

from ipysigma import Sigma

Graph = build_dep_dest_graph(final_pointcalls_navigo, final_pointcalls_in, 'pointcall', 'pointcall')
Sigma(Graph, start_layout=True)

In [ ]:
import pandas as pds

# Load an empty map
from keplergl import KeplerGl

kepler_base_conf = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

In [ ]:
ports_dest_french_flag = {}
ports_dest_ext_flag = {}

for p in final_pointcalls_in:
    port = p['pointcall']
    french_flag = p['ship_flag_standardized_fr'] in ['français'] 
    tonnage = int(p['tonnage']) if p['tonnage'] is not None else 0
    
    if french_flag:
        if port not in ports_dest_french_flag:
            lat = float(p['latitude']) if p['latitude'] is not None else None
            lon = float(p['longitude']) if p['longitude'] is not None else None
            ports_dest_french_flag[port] = {
                'latitude_dep':45.81666,
                'longitude_dep':-1.116667, # je schématise en centralisant tous les departs sur le port de Marennes, alors qu'en réalité certains départs se font depuis d'autres port du bureau des Fermes de Marennes (--> Ribérou)
                'nb_pointcalls': 1,
                'latitude_dest': lat,
                'longitude_dest': lon,
                'sous_etat': p['substate_1789_fr'],
                'etat': p['state_1789_fr'],
                'tonnages':tonnage,
            }
        else:
            ports_dest_french_flag[port]['nb_pointcalls'] += 1
            ports_dest_french_flag[port]['tonnages'] += tonnage
    else:
        if port not in ports_dest_ext_flag:
            lat = float(p['latitude']) if p['latitude'] is not None else None
            lon = float(p['longitude']) if p['longitude'] is not None else None
            ports_dest_ext_flag[port] = {
                'latitude_dep':45.81666,
                'longitude_dep':-1.116667, 
                'nb_pointcalls': 1,
                'latitude_dest': lat,
                'longitude_dest': lon,
                'sous_etat': p['substate_1789_fr'],
                'etat': p['state_1789_fr'],
                'tonnages':tonnage,
            }
        else:
            ports_dest_ext_flag[port]['nb_pointcalls'] += 1
            ports_dest_ext_flag[port]['tonnages'] += tonnage
            
ports_dest_french_flag = [{'port': port, **vals} for port, vals in ports_dest_french_flag.items()]
ports_dest_ext_flag = [{'port': port, **vals} for port, vals in ports_dest_ext_flag.items()]

# print("french flags : ", ports_dest_french_flag[0:10])
# print("\n\next flags : ", ports_dest_ext_flag[0:20])

with open('assets/results_french.json', 'w') as fp1:
    json.dump(ports_dest_french_flag, fp1)
with open('assets/results_ext.json', 'w') as fp2:
    json.dump(ports_dest_ext_flag, fp2)

In [ ]:
# carte finale en distinguant les trajets effectués par des bateaux au pavillon français (en bleu) / etrangers (en orange)
# la taille des flux dépend du tonnage cumulé
# je schématise en centralisant tous les departs sur le port de Marennes, alors qu'en réalité certains départs se font depuis d'autres port du bureau des Fermes de Marennes (--> Ribérou)

salt_out_Marennes_1789_map = KeplerGl(config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "l0a783h",
          "type": "point",
          "config": {
            "dataId": "khfjp7ya8",
            "label": "destinations des pavillons français",
            "color": [
              41,
              76,
              181
            ],
            "columns": {
              "lat": "latitude_dest",
              "lng": "longitude_dest",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                5.7,
                58
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": []
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": {
              "name": "tonnages",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        },
        {
          "id": "vma3uej",
          "type": "point",
          "config": {
            "dataId": "8zff2618",
            "label": "destinations des pavillons étrangers",
            "color": [
              215,
              63,
              13
            ],
            "columns": {
              "lat": "latitude_dest",
              "lng": "longitude_dest",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                5.7,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": {
              "name": "tonnages",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "khfjp7ya8": [
              {
                "name": "port",
                "format": None
              },
              {
                "name": "nb_pointcalls",
                "format": None
              },
              {
                "name": "sous_etat",
                "format": None
              },
              {
                "name": "etat",
                "format": None
              },
              {
                "name": "tonnages",
                "format": None
              }
            ],
            "8zff2618": [
              {
                "name": "port",
                "format": None
              },
              {
                "name": "nb_pointcalls",
                "format": None
              },
              {
                "name": "sous_etat",
                "format": None
              },
              {
                "name": "etat",
                "format": None
              },
              {
                "name": "tonnages",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 44.241525922142564,
      "longitude": -58.62994515418118,
      "pitch": 0,
      "zoom": 2.007752247509027,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}, data={
    'khfjp7ya8': pds.DataFrame(ports_dest_french_flag),
    '8zff2618': pds.DataFrame(ports_dest_ext_flag)
    })

salt_out_Marennes_1789_map